In [5]:
import numpy as np
import cv2
import os
from glob import glob
import matplotlib.pyplot as plt
from PIL import Image

Image.MAX_IMAGE_PIXELS = None  # Disable decompression bomb check

In [6]:
# 1. Define file paths
image_folder = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco van Nes\WvN016_546_strategy2_20250308\test_run_5x5_20250308at095255\WhiteLight"
roi_file = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco van Nes\WvN016_546_strategy2_20250308\test_run_5x5_20250308at095255\ROIs5x5.txt"

# 2. Load ROI coordinates (assumes format: y x)
txt = np.loadtxt(roi_file, dtype=int)

# 3. Extract unique X and Y positions to determine grid size
x_pos = np.unique(txt[:, 1])  # Unique X positions (columns)
y_pos = np.unique(txt[:, 0])  # Unique Y positions (rows)

N_cols = len(x_pos)  # Number of columns
N_rows = len(y_pos)  # Number of rows

# 4. Load image file paths and sort them
image_files = sorted(glob(os.path.join(image_folder, "*.tif")))

# 5. Assume all images have the same resolution (e.g., 5120x5120)
sample_img = Image.open(image_files[0])  # Use PIL to avoid OpenCV memory errors
img_h, img_w = sample_img.size  # Get image dimensions

# 6. Create an empty canvas for the stitched image (now in uint8)
stitched_img = np.zeros((N_rows * img_h, N_cols * img_w), dtype=np.uint8)

# 7. Loop through each image and place it correctly
for i in range(len(txt)):
    # Load image using PIL to handle large files
    img = Image.open(image_files[i])
    img = np.array(img, dtype=np.uint16)  # Convert to NumPy array

    # Convert 16-bit to 8-bit
    img = cv2.convertScaleAbs(img, alpha=(255/65535))

    x, y = txt[i, 1], txt[i, 0]  # Get ROI coordinates

    idx_x = np.where(x_pos == x)[0][0]  # Find column index
    idx_y = np.where(y_pos == y)[0][0]  # Find row index

    # 8. Place the image into the correct position in the stitched image
    stitched_img[idx_y * img_h : (idx_y + 1) * img_h, idx_x * img_w : (idx_x + 1) * img_w] = img

    print(f"Placed image {i+1}/{len(txt)} at position ({idx_y}, {idx_x})")

# 9. Display the final stitched image
plt.imshow(stitched_img, cmap="gray")
plt.axis("off")
plt.show()

# 10. Save the stitched image
output_path = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco van Nes\WvN016_546_strategy2_20250308\test_run_5x5_20250308at095255\stitched_image_uint8.tif"
cv2.imwrite(output_path, stitched_img)
print(f"Stitched image saved at {output_path}")

MemoryError: Unable to allocate 352. GiB for an array with shape (614400, 614400) and data type uint8